## 한국은행 기준금리

In [1]:
import re
import requests
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

In [97]:
url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'
resp = requests.get(url)
soup = BeautifulSoup(resp.content,'lxml')
td_tag = soup.select('div.table.tac tr td')
context = [td.text for td in td_tag]

dic_baserate = {}
dic_baserate['변경년도'],dic_baserate['변경월일'],dic_baserate['기준금리'] = [], [] ,[]

for i in range(int(len(context)/3)):
  dic_baserate['변경년도'].append(context[i*3])
  dic_baserate['변경월일'].append(context[i*3+1])
  dic_baserate['기준금리'].append(float(context[i*3+2]))

dic_baserate['날짜'] = []
for i in range(len(dic_baserate['변경월일'])):
  dic_baserate['날짜'].append(dic_baserate['변경년도'][i]+'-'+dic_baserate['변경월일'][i].replace('월','').replace('일','').replace(' ','-'))

base_rate = pd.DataFrame(dic_baserate)
cols = list(base_rate)
base_rate = base_rate.loc[5:33][cols[2:]] #2004년 11월부터 2017년 12월까지

In [98]:
dic_date = {}
dic_date['날짜'] = pd.date_range("2004-11-11","2017-12-31").astype(str)
date = pd.DataFrame(dic_date)

data = date.merge(base_rate,how='outer')
for i in range(len(data)-1):
    if pd.isna(data.iloc[i,1]) == False and pd.isna(data.iloc[i+1,1]) == True:
        data.iloc[i+1,1] = data.iloc[i,1]

data = data.iloc[171:] #2005년 5월부터

In [99]:
data.to_csv('base_rate.csv',index=False)